In [85]:
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import requests

In [36]:
#declaring base open and colsed docket link
open_docket_base_link = "https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0#s-skipLinkTargetForMainSearchResults"
closed_docket_base_link = "https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1#s-skipLinkTargetForMainSearchResults"

In [105]:
#declaring chrome header
hdr1 = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

def html_data_grabber(given_page_link,header = hdr1) : #having this function for ease if we need proxies implementation in future
    request_made = requests.get(given_page_link,headers = header)
    status_code = int(request_made.status_code)
    data = ""
    grabbed_data = False
    if status_code == 200 :
        grabbed_data = True
        data = request_made.text
        message = "Success"
    elif status_code >= 403 :
        message = "banned"
    elif status_code >= 500 :
        message = "Government Server is down"
    else :
        message = "your internet connection is down"
        
    if grabbed_data :
        data = soup(data,"html.parser")
        
    return {"content":data, "status_code":status_code, "grabbed_data":grabbed_data, "message": message}
        
def available_search_result_page_links(base_link) :
    #this function returns list of all search result pages link that can be browsed from base page
    base_link_content = html_data_grabber(base_link)
    if base_link_content["grabbed_data"] :
        last_page_link = "https://www.gao.gov/legal/bid-protests/search" + base_link_content["content"].findAll("a",{"title" : "Go to last page"})[0]["href"]
    else :
        print(base_link,base_link_content["message"],base_link_content["status_code"])
        return False

    last_page_no = int(last_page_link.split("&")[-1].replace("page=",""))
    all_links = [last_page_link.split("page=")[0] + "page=" + str(i) for i in range(0,last_page_no + 1)]
    return all_links

def docket_page_links_from_search_page(page_link) :
    browse_link = html_data_grabber(page_link)
    if browse_link["grabbed_data"] :
        docket_content = browse_link["content"].findAll("article",{"class" : "node node--type-bid-protest-docket node--view-mode-teaser-search"})
    else :
        print(page_link,browse_link["message"],browse_link["status_code"])
        return False
    all_links = []
    base_link = "https://www.gao.gov"
    all_links = [base_link + i["about"] for i in docket_content]
    return all_links

def extract_data_from_docket_page(docket_page_link) :
    browse_link = html_data_grabber(docket_page_link)
    if browse_link["grabbed_data"] :
        data = browse_link["content"].findAll("div",{"class" : "node__content"})[0]
    else :
        print(docket_page_link, browse_link["message"],browse_link["status_code"])
        return False
    all_data = {"docket_page_link" : docket_page_link}
    all_data_lines = data.findAll("div",{"class" : re.compile(r'^field field--')})
    for single_data in all_data_lines :
        try :
            field_name = single_data.findAll("header",{"class" : "field__label"})[0].text.strip()
        except :
            field_name = "error"
        try :
            field_value = single_data.findAll("div",{"class" : "field__item"})[0].text.strip()
        except :
            field_value = "error"
        all_data[field_name] =  field_value
    return all_data

def all_docket_data_from_search_page_link(search_page_link) :
    all_docket_page_links = docket_page_links_from_search_page(search_page_link)
    all_docket_data = [extract_data_from_docket_page(i) for i in all_docket_page_links]
    return pd.DataFrame(all_docket_data)
        



In [120]:
available_search_result_page_links("https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=3")

['https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=0',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=1',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=2',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=3',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=4',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=5',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=6',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=7',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=8',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=9',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=10',
 'https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=11']

In [99]:
docket_page_links_from_search_page("https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=39")

['https://www.gao.gov/docket/b-419682.2',
 'https://www.gao.gov/docket/b-419369.3',
 'https://www.gao.gov/docket/b-419784.1',
 'https://www.gao.gov/docket/b-419780.1',
 'https://www.gao.gov/docket/b-419695.1',
 'https://www.gao.gov/docket/b-418400.8',
 'https://www.gao.gov/docket/b-419682.1',
 'https://www.gao.gov/docket/b-418400.7',
 'https://www.gao.gov/docket/b-419539.2',
 'https://www.gao.gov/docket/b-419539.1',
 'https://www.gao.gov/docket/b-419542.1',
 'https://www.gao.gov/docket/b-419268.2',
 'https://www.gao.gov/docket/b-419552.1',
 'https://www.gao.gov/docket/b-419728.1',
 'https://www.gao.gov/docket/b-419755.1',
 'https://www.gao.gov/docket/b-419655.1',
 'https://www.gao.gov/docket/b-419778.1',
 'https://www.gao.gov/docket/b-419552.3',
 'https://www.gao.gov/docket/b-419680.1',
 'https://www.gao.gov/docket/b-419766.1']

In [121]:
extract_data_from_docket_page("https://www.gao.gov/docket/b-420105.3")

{'docket_page_link': 'https://www.gao.gov/docket/b-420105.3',
 'Protestor': 'Intellectix Corporation',
 'Solicitation Number': 'N/A',
 'Agency': 'Department of Justice : Drug Enforcement Administration',
 'File number': 'B-420105.3',
 'Status': 'Case Currently Open',
 'Filed Date': 'Sep 17, 2021',
 'Due Date': 'Dec 27, 2021',
 'Case Type': 'Bid Protest',
 'GAO Attorney': 'Michael P. Grogan'}

In [122]:
all_docket_data_from_search_page_link("https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=3")

,docket_page_link,Protestor,Solicitation Number,Agency,File number,Status,Filed Date,Due Date,Case Type,GAO Attorney
0,https://www.gao.gov/docket/b-420176.1,"PrimCorp, LLC",FA4484-21-R-0003,Department of the Air Force,B-420176.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,David A. Edelstein
1,https://www.gao.gov/docket/b-420168.1,Vectrus Systems Corporation,HC102819R0009,Department of Defense : Defense Information Sy...,B-420168.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,Glenn G. Wolcott
2,https://www.gao.gov/docket/b-420170.1,"Basic Concepts, Inc.",SPE4A7-21-Q-1235,Department of Defense : Defense Logistics Agency,B-420170.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,Paul N. Wengert
3,https://www.gao.gov/docket/b-420171.1,"Chenega Enterprise Systems & Solutions, LLC",RFQ1509663,Department of Agriculture : Natural Resources ...,B-420171.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,Christine Milne
4,https://www.gao.gov/docket/b-420175.1,"Ad Hoc, LLC",RFQ-CMS-2021-210887,Department of Health and Human Services : Cent...,B-420175.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,Kenneth Kilgour
5,https://www.gao.gov/docket/b-420177.1,AT&T Corporation,15F06720R0000174-TO1,Department of Justice : Federal Bureau of Inve...,B-420177.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,Jonathan L. Kang
6,https://www.gao.gov/docket/b-420178.1,AT&T Corporation,15F06720R0000174-TO2,Department of Justice : Federal Bureau of Inve...,B-420178.1,Case Currently Open,"Sep 20, 2021","Dec 29, 2021",Bid Protest,Jonathan L. Kang
7,https://www.gao.gov/docket/b-419024.5,"A. Prentice Ray & Associates, LLC",GS00Q-13-DR-0002-8a,Independent Agencies and Govt Corporations : F...,B-419024.5,Case Currently Open,"Sep 17, 2021","Dec 27, 2021",Bid Protest,Charmaine Stevenson
8,https://www.gao.gov/docket/b-420045.2,"W.W. Grainger, Inc.",47QSCC-20-Q-5015,Independent Agencies and Govt Corporations : F...,B-420045.2,Case Currently Open,"Sep 17, 2021","Dec 27, 2021",Bid Protest,U R. Yoo
9,https://www.gao.gov/docket/b-420105.3,Intellectix Corporation,N/A,Department of Justice : Drug Enforcement Admin...,B-420105.3,Case Currently Open,"Sep 17, 2021","Dec 27, 2021",Bid Protest,Michael P. Grogan


In [ ]:
#due : implement crawl resume method
#resume from where it left if crawing interrupts midway

In [ ]:
Driver Code

In [110]:
#grabbing data of all open docket
file_count = 0
all_open_docket_data = []
all_search_pages_results = available_search_result_page_links(open_docket_base_link)
for single_link in all_search_pages_results :
    print(single_link)
    data = all_docket_data_from_search_page_link(single_link)
    all_open_docket_data.append(data)
    
    file_count = file_count + 1
    percent_calci = (float(file_count)/float(len(all_search_pages_results)))*100.0
    print ("working on link no " + str(file_count) + " of total links " + str(len(all_search_pages_results)) + " " + str(percent_calci) + " % completed")

https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=0
working on link no 1 of total links 12 8.333333333333332 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=1
working on link no 2 of total links 12 16.666666666666664 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=2
working on link no 3 of total links 12 25.0 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=3
working on link no 4 of total links 12 33.33333333333333 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=4
working on link no 5 of total links 12 41.66666666666667 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=5
working on link no 6 of total links 12 50.0 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=0&page=6
working on link no 7 of total links 12 58.333333333333336 % completed
https://www.gao.gov/leg

In [114]:
df_open_docket = pd.concat(all_open_docket_data)

In [123]:
df_open_docket

,docket_page_link,Protestor,Solicitation Number,Agency,File number,Status,Filed Date,Due Date,Case Type,GAO Attorney
0,https://www.gao.gov/docket/b-420206.1,VidmanBarber LLC,N/A,Department of Education,B-420206.1,Case Currently Open,"Sep 29, 2021","Jan 7, 2022",Bid Protest,Peter H. Tran
1,https://www.gao.gov/docket/b-420203.1,"ICU Medical Sales, Inc.",36C25721Q1121,Department of Veterans Affairs,B-420203.1,Case Currently Open,"Sep 29, 2021","Jan 7, 2022",Bid Protest,Paul N. Wengert
2,https://www.gao.gov/docket/b-420205.1,"Cherokee CRC, LLC",140A1621R0062,Department of the Interior : Bureau of Indian ...,B-420205.1,Case Currently Open,"Sep 29, 2021","Jan 7, 2022",Bid Protest,Peter H. Tran
3,https://www.gao.gov/docket/b-419956.24,"CWS FMTI JV, LLC",75N98121R00001,Department of Health and Human Services : Nati...,B-419956.24,Case Currently Open,"Sep 29, 2021","Jan 7, 2022",Bid Protest,Jonathan L. Kang
4,https://www.gao.gov/docket/b-420207.1,"Video Dynamics, Inc.",HHM402-20-R-0029,Department of Defense : Defense Intelligence A...,B-420207.1,Case Currently Open,"Sep 29, 2021","Jan 7, 2022",Bid Protest,Christina Sklarew
...,...,...,...,...,...,...,...,...,...,...
16,https://www.gao.gov/docket/b-419705.2,"Academy Leadership, LLC",70CMSD21R00000001,Department of Homeland Security : United State...,B-419705.2,Case Currently Open,"Jun 22, 2021","Sep 30, 2021",Bid Protest,Young H. Cho
17,https://www.gao.gov/docket/b-419258.4,"GOV National Healthcare Drive, LLC",36C10F18R0659,Department of Veterans Affairs,B-419258.4,Case Currently Open,"May 14, 2021","Aug 23, 2021",Bid Protest: Cost,Scott H. Riback
18,https://www.gao.gov/docket/b-419829.2,"RCH Partners, LLC",503102-21-Q-0012,Independent Agencies and Govt Corporations : U...,B-419829.2,Case Currently Open,"May 13, 2021","Aug 23, 2021",Bid Protest,Heather Self
19,https://www.gao.gov/docket/b-418631.7,"Apex Transit Solutions, LLC",36C25020Q0093,Department of Veterans Affairs,B-418631.7,Case Currently Open,"Apr 6, 2021","Jul 15, 2021",Bid Protest,Katherine I. Riback


In [115]:
df_open_docket.to_csv("open_docket.csv",index = False)

In [111]:
#grabbing data of all closed docket
file_count = 0
all_closed_docket_data = []
all_search_pages_results = available_search_result_page_links(closed_docket_base_link)
for single_link in all_search_pages_results :
    print(single_link)
    data = all_docket_data_from_search_page_link(single_link)
    all_closed_docket_data.append(data)
    
    file_count = file_count + 1
    percent_calci = (float(file_count)/float(len(all_search_pages_results)))*100.0
    print ("working on link no " + str(file_count) + " of total links " + str(len(all_search_pages_results)) + " " + str(percent_calci) + " % completed")

https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=0
working on link no 1 of total links 101 0.9900990099009901 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=1
working on link no 2 of total links 101 1.9801980198019802 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=2
working on link no 3 of total links 101 2.9702970297029703 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=3
working on link no 4 of total links 101 3.9603960396039604 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=4
working on link no 5 of total links 101 4.9504950495049505 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=5
working on link no 6 of total links 101 5.9405940594059405 % completed
https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=6
working on link no 7 of total links 101 6.93069306930693

ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

In [116]:
df_close_docket = pd.concat(all_closed_docket_data)

In [117]:
df_close_docket.to_csv("closed_docket.csv",index = False)

In [118]:
df_close_docket

,docket_page_link,Protestor,Solicitation Number,Agency,File number,Outcome,Decision Date,Filed Date,Due Date,Case Type,GAO Attorney,Decision Summary
0,https://www.gao.gov/docket/b-420002.1,First American Business Solutions,W91RUS21R0078,Department of the Army : U.S. Army Materiel Co...,B-420002.1,Denied,"Sep 29, 2021","Jul 21, 2021","Oct 29, 2021",Bid Protest,Emily R. O'Hara,"This protest is covered by a protective order,..."
1,https://www.gao.gov/docket/b-420117.1,Centuria Corporation,80KSC021Q0007,Independent Agencies and Govt Corporations : N...,B-420117.1,Dismissed,"Sep 29, 2021","Aug 30, 2021","Dec 8, 2021",Bid Protest,Hannah G. Barnes,NaN
2,https://www.gao.gov/docket/b-420125.1,"Pitney Bowes, Inc.",RFQ1503994,Department of the Army,B-420125.1,Dismissed,"Sep 29, 2021","Sep 2, 2021","Dec 13, 2021",Bid Protest,Christine Milne,NaN
3,https://www.gao.gov/docket/b-420164.1,"Hawk Contracting Group, LLC",36C77621R0149,Department of Veterans Affairs,B-420164.1,Dismissed,"Sep 29, 2021","Sep 17, 2021","Dec 27, 2021",Bid Protest,Christine Milne,NaN
4,https://www.gao.gov/docket/b-420102.1,"Facility Services Management, Inc.",W912DY21FXXXX,Department of the Army : Corps of Engineers,B-420102.1,Dismissed,"Sep 28, 2021","Aug 27, 2021","Dec 6, 2021",Bid Protest,Katherine I. Riback,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15,https://www.gao.gov/docket/b-418626.6,"Zolon PCS, LLC",28321320Q00000011,Independent Agencies and Govt Corporations : S...,B-418626.6,Dismissed,"Oct 15, 2020","Sep 28, 2020","Jan 6, 2021",Bid Protest,April Y. Shields,NaN
16,https://www.gao.gov/docket/b-418976.1,"ThREE Consulting, LLC",Unknown,Department of the Army : U.S. Army Materiel Co...,B-418976.1,Dismissed,"Oct 15, 2020","Aug 3, 2020","Nov 12, 2020",Bid Protest,Paula A. Williams,NaN
17,https://www.gao.gov/docket/b-419159.1,The Povolny Group,36E77620R0030,Department of Veterans Affairs,B-419159.1,Dismissed,"Oct 15, 2020","Sep 21, 2020","Dec 30, 2020",Bid Protest,Michael P. Grogan,NaN
18,https://www.gao.gov/docket/b-418773.2,"M6-Vets, LLC",9531CB19Q0112,Independent Agencies and Govt Corporations : C...,B-418773.2,Dismissed,"Oct 14, 2020","Sep 11, 2020","Dec 21, 2020",Bid Protest,Glenn G. Wolcott,NaN


In [119]:
import os
os.getcwd()

'/Users/ujjawalnarayan/chromedriver/backup_notebook'

In [ ]:
2021/10/1/open.csv.187921
2021/10/1/close.csv.187921

In [ ]:
crwal.py -all -path